In [82]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, Image
import pandas as pd
from PIL import Image
import math

In [144]:
# def divide_img_blocks(img, n_blocks=(3, 3)):
#   horizontal = np.array_split(img, n_blocks[0])
#   splitted_img = [np.array_split(block, n_blocks[1], axis=1) for block in horizontal]
#   reshaped_images = []

#   out_w = math.ceil(img.shape[0] / 3)
#   out_h = math.ceil(img.shape[1] / 3)

#   # out_images = []
#   # for im_group in splitted_img:
#   #   for im in im_group:
#   #     im = im[:out_w-1, :out_h-1, :]
#   #     out_images.append(im)
#   print(out_w, out_h)
#   for sub_images in splitted_img:
#     for sub_image in sub_images:
#       reshaped_image = sub_image.reshape(sub_image.shape[0], sub_image.shape[1], sub_image.shape[2])
#       reshaped_image = reshaped_image[:out_w-1, :out_h-1, :]
#       print(reshaped_image.shape)

#       reshaped_images.append(reshaped_image.T)

#   # Convert the list of arrays into a NumPy array
#   out_images = np.asarray(reshaped_images).T
#   return out_images
#   # out_images = np.asarray(splitted_img, dtype=np.ndarray).reshape((3, 3))
#   # return out_images.T

def get_split_and_new_coords(image_width, image_height, x, y):
  """
  This function takes the image dimensions, x-coordinate, and y-coordinate and returns:
  - The split number (0-5) the coordinate belongs to.
  - The new x and y coordinates within that split.

  Args:
    image_width: Width of the image.
    image_height: Height of the image.
    x: X-coordinate of the point.
    y: Y-coordinate of the point.
  """

  # Calculate the width and height of each split
  split_width = image_width // 3
  split_height = image_height // 3

  # Determine the x and y split based on the coordinates
  x_split = (x // split_width)
  y_split = (y // split_height)

  # Calculate the new x and y coordinates within the split
  new_x = x % split_width
  new_y = y % split_height

  return x_split, new_x, y_split, new_y


In [4]:
source_file = 'training_elephants_gsd.csv'

df_coords = pd.read_csv(source_file, header=None, skiprows=1, names=['fname', 'x', 'y'])

In [145]:
completed_names = []
split_coord_outcomes = []

for i in range(0, df_coords.shape[0]):
    if i % 100 == 0:
        print(f"Completed {i} images")

    image_name, center_x, center_y = df_coords.iloc[i, 0:3].values
    if image_name in completed_names:
        continue
    
    # Splitting image into 9 parts.
    df_coords_im = df_coords[df_coords.fname == image_name].values
    image = cv2.imread("gsd_training_images/" + image_name + ".jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image.shape[0:2]

    # Calculate the width and height of each split
    split_width = w // 3
    split_height = h // 3
    i = 0

    gsd_dir = "gsd_training_images/"
    split_images_dir = 'split_images_gsd/'
    img = Image.open(gsd_dir + image_name + '.jpg')
    metadata = img.getexif()

    # Iterate through the rows and columns to create and save split images
    for row in range(3):
        for col in range(3):
            # Extract the current split image
            split_image = image[row * split_height:(row + 1) * split_height,
                        col * split_width:(col + 1) * split_width]

            new_path = split_images_dir + image_name + "_" + str(col) + str(row) + ".jpg"

            img_pil = Image.fromarray(split_image)
            img_pil.save(new_path, exif=metadata)
            i += 1

    # Splitting coordinates and translating them appropriately.
    split_coords = []
    for item in df_coords_im:
        x, y = item[1], item[2]
        split_img_x, new_x, split_img_y, new_y = get_split_and_new_coords(
            w, h, x, y)
        split_coords.append((split_img_x, split_img_y, new_x, new_y))

    for i in range(0, 3):
        for j in range(0, 3):
            this_split_coords = []
            for item in split_coords:
                split_x, split_y = item[0:2]
                if i == split_x and j == split_y:
                    this_split_coords.append((item[2:4]))
                    
            for split in this_split_coords:
                split_coord_outcomes.append([image_name + "_" + str(i) + str(j), split[0], split[1]])

    completed_names.append(image_name)

Completed 0 images
Completed 100 images
Completed 200 images
Completed 300 images
Completed 400 images
Completed 500 images
Completed 600 images
Completed 700 images
Completed 800 images
Completed 900 images
Completed 1000 images
Completed 1100 images
Completed 1200 images
Completed 1300 images
Completed 1400 images
Completed 1500 images
Completed 1600 images
Completed 1700 images
Completed 1800 images
Completed 1900 images
Completed 2000 images
Completed 2100 images
Completed 2200 images
Completed 2300 images
Completed 2400 images
Completed 2500 images
Completed 2600 images
Completed 2700 images
Completed 2800 images
Completed 2900 images
Completed 3000 images
Completed 3100 images
Completed 3200 images
Completed 3300 images
Completed 3400 images
Completed 3500 images
Completed 3600 images
Completed 3700 images
Completed 3800 images
Completed 3900 images
Completed 4000 images
Completed 4100 images
Completed 4200 images
Completed 4300 images
Completed 4400 images
Completed 4500 images


In [153]:
df_split_coords = pd.DataFrame(split_coord_outcomes, columns=['image_name', 'x', 'y'])
print(df_split_coords.head)

df_split_coords.to_csv('training_elephants_split.csv', index=False, header=None)

<bound method NDFrame.head of                                         image_name    x     y
0      2aada86263161acc6e207ea329759556d308458a_00  478     2
1      8f30778719e5af075797f994273603c94f170e29_00   74   168
2      8f30778719e5af075797f994273603c94f170e29_02  804   724
3      8f30778719e5af075797f994273603c94f170e29_02  756   754
4      8f30778719e5af075797f994273603c94f170e29_02  654   720
...                                            ...  ...   ...
12600  0d68ad29d15049050d8f446bb89bc0c790ce651a_02  435   419
12601  0d68ad29d15049050d8f446bb89bc0c790ce651a_02  235   333
12602  b50380bbaf285cce160e06d4fe3791947cfdd6bc_22  391   921
12603  b50380bbaf285cce160e06d4fe3791947cfdd6bc_22  440   942
12604  b50380bbaf285cce160e06d4fe3791947cfdd6bc_22  542  1037

[12605 rows x 3 columns]>
